Title 

In [1]:
library(tidyverse) 
library(tidymodels)
library(gridExtra)
options(repr.matrix.max.rows = 6)
url <-"https://raw.githubusercontent.com/katelynmilan/DSCI_project/main/processed.cleveland.data"
data_set <- read_csv(url, col_names = c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num"))
data_set

Warning message:
“package ‘ggplot2’ was built under R version 4.3.2”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_co

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3
57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0,1
38,1,3,138,175,0,0,173,0,0.0,1,?,3.0,0


In [3]:
clean_data <- data_set|> 
select(age, chol, num) |> 
rename(Heart_Disease = num, Cholesterol = chol, Age = age)|>
mutate(Heart_Disease = as.character(Heart_Disease))   

data_split <- initial_split(clean_data, prop = 0.75, strata = Heart_Disease)
data_train <- training(data_split)
data_test <- testing(data_split)

data_train 

Age,Cholesterol,Heart_Disease
<dbl>,<dbl>,<chr>
37,250,0
41,204,0
56,236,0
⋮,⋮,⋮
61,166,4
55,205,3
59,176,3


In [ ]:
credit_recipe2 <- recipe(Balance ~ Income + Rating, data = credit_training)|>
                    step_scale(all_predictors())|>
                    step_center(all_predictors())

knn_rmspe <- nearest_neighbor(weight_func = "rectangular", neighbors = tune())|>
            set_engine("kknn")|>
            set_mode("regression")

credit_workflow <- workflow()|>
                    add_model(knn_rmspe)|>
                    add_recipe(credit_recipe2)

credit_vfold<- vfold_cv(credit_training, v = 5, strata = Balance)

grid <- tibble(neighbors = seq(from = 1, to = 10, by = 1))

best_k <- credit_workflow |>
            tune_grid(resamples = credit_vfold, grid = grid)|>
            collect_metrics()|>
            filter(.metric == "rmse")|>
            slice_min(mean, n = 1)|>
            pull(neighbors)

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k)|>
            set_engine("kknn")|>
            set_mode("regression")

knn_fit <- workflow()|>
            add_model(knn_spec)|>
            add_recipe(credit_recipe2)|>
            fit(data = credit_training)

credit_pred <- knn_fit |> 
           predict(credit_testing) |>
           bind_cols(credit_testing) |>
           metrics(truth = Balance, estimate = .pred)|>
            filter(.metric == "rmse")|>
            select(.estimate)|>
            pull()

credit_pred